# 18-Oct-2022

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Image, HTML, display

## Summary of Goals
1. Some details of DMD from the literature
2. Computing the norm of the error in the update (error introduced by the SVD truncation and approximation of the DMD algorithm)
3. Added timing to code, and fixed an algorithm bug in the matrix multiplication

## 1. DMD in the literature
### 1.1 Limitations of DMD
SVD-based approaches are inefficient to handle invariance in the data
examples:
1. Translational and rotational invariances ==> DMD predicts a artificially high dimensional space for modes<br><br>
 <img src="DMDLimits_translational.png" width=800 alt="DMD limits for added translation"><br><br>
 It can be seen from the picture below that although we have 2 physical modes, DMD suggests using at least 10 modes.<br><br>
   <img src="DMDLimits_translational_numModes.png" width=800 alt="DMD limits for added translation - number of modes">



2. Transient time behaviour ==> DMD fails. <br>Example is a mode that turns on and off.

## 2. Uncertainty in our update
I think there are 3 error norms that we can compute:
1. Computing norm by computing the full-dimensional evolution A
    * This doesn't tell us about the error introduced by the SVD approximation
    <pre><code>A = X2@V@np.linalg.inv(S_matrix)@U.T
   diff = X2 - A@X1
   np.linalg.norm(diff, 'fro')</code></pre>

2. Computing the error by projecting Atilde back onto the full-dimensional space
    * computationally expensive
    <pre><code>diff = X2 - Ur@Atilde@Ur.T@X1
   np.linalg.norm(diff, 'fro')</code></pre>


3. Computing the error by projecting the X and X' matrices onto X's POD modes
    * the error is smaller than the <strong>2nd</strong> method
    <pre><code>diff = Ur.T@X2 - Atilde@(Ur.T@X1)
   np.linalg.norm(diff, 'fro')</code></pre>
    

The plots below shows the Frobenius norm for method 3 above.

In [17]:
display(HTML("<table><tr><td><img src='NavStrW5H1S10UpdateNorms.png' width=650></td><td><img src='PoissonS10UpdateNorms.png' width=700></td></tr></table>"))

,


From the plots above, we can see that for larger iterations we get a smaller norm for the DMD update.

In [18]:
display(HTML("<table><tr><td><img src='NavStrW5H1S10MultiUpdateNorms.png' width=650></td><td><img src='PoissonMultiupdateupdateNorms.png' width=700></td></tr></table>"))

,


As can be seen from the figures above, when we have multiple updates, the 2nd update has a smaller norm although it might not be as effective.<br>
This inconsisstency makes it harder for us to use this norm to measure effectiveness (uncertainty) of our correction.

## 3. Code timings: Matrix chain multiplication
Based on the timings, computing matrix multiplications took the longest time. This was resolved by solving a matrix chain ordering problem.<br>
Before, the order of operations was:
$$ G = U_r \tilde{G} U_r^{\intercal} $$
$$ update = G \mathbf{x}^\prime_{m-1} $$
<span style="color: #0e6655 ">The order for this multiplication: $O(nr^2) + O(n^2 r) + O(n^2)$ </span><br>
I chnaged it with: 
$$ \mathbf{\tilde{x}}^\prime_{m-1} = U_r^{\intercal} \mathbf{x}^\prime_{m-1} $$
$$ update = U_r^{\intercal} \tilde{G} \mathbf{\tilde{x}}^\prime_{m-1} $$

<span style="color:  #0e6655 ;">The order for this multiplication: $ O(rn) + O(nr^2) + O(nr) $</span><br>
This reduced the matrix multiplication time from, for example: (result from Python)
- $ 0.99737 s$ to $0.10016 s$ 